### Setup a Conda Environment on Coiled

In [ ]:
import coiled

coiled.create_software_environment(
   name='mapshader-tiling',
   conda={
       'channels': ['conda-forge', 'defaults'],
       'dependencies': [
           'python=3.9',
           'mapshader',
           'dask=2022.04.2',
           'distributed=2022.4.2',
           'cloudpickle=2.0.0',
           'spatialpandas',
           'boto3',
       ],
    },
)

### Create Dask Cluster on Coiled 

In [ ]:
from coiled.v2 import Cluster
cluster = Cluster(name='mapshader-tiler',
                  n_workers=10,
                  worker_cpu=2,
                  worker_options={"nthreads": 1},
                  scheduler_memory="8 GiB",
                  software='mapshader-tiling')

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

### Clear cluster memory if necessary

In [ ]:
client.restart()

## Tile World Cities (Sparse Points)

### Setup Mapshader Source

In [ ]:
import geopandas as gpd
import mapshader
import spatialpandas

from mapshader.sources import VectorSource


def world_cities_source():

    # construct transforms
    reproject_transform = dict(name='reproject_vector', args=dict(epsg=3857))
    add_xy_fields_transform = dict(name='add_xy_fields', args=dict(geometry_field='geometry'))
    buffered_extent_transform = dict(name='add_projected_buffered_extent', 
                                     args=dict(crs='4326',
                                               buffer_distance=.01,
                                               geometry_field='geometry'))
    sp_transform = dict(name='to_spatialpandas', args=dict(geometry_field='geometry'))
    
    transforms = [reproject_transform,
                  add_xy_fields_transform,
                  buffered_extent_transform,
                  sp_transform]

    # construct value obj
    source_obj = dict()
    source_obj['name'] = 'World Cities'
    source_obj['key'] = 'world-cities'
    source_obj['text'] = 'World Cities'
    source_obj['description'] = 'World Cities Point Locations'
    source_obj['geometry_type'] = 'point'
    source_obj['agg_func'] = 'max'
    source_obj['cmap'] = ['aqua', 'aqua']
    source_obj['shade_how'] = 'linear'
    source_obj['dynspread'] = 2
    source_obj['raster_interpolate'] = 'linear'
    source_obj['xfield'] = 'X'
    source_obj['yfield'] = 'Y'
    source_obj['filepath'] = gpd.datasets.get_path('naturalearth_cities')
    source_obj['transforms'] = transforms
    source_obj['service_types'] = ['tile', 'wms', 'image', 'geojson']

    return source_obj


cities_source = VectorSource.from_obj(world_cities_source())
cities_source.load()
cities_source.data

### Example Mapshader to AWS S3 Helper Function

In [ ]:
from mapshader.core import render_map
from mapshader.sources import MapSource
import os
import sys
from PIL.Image import fromarray
import numpy as np

from io import BytesIO

def to_s3_tile(source: MapSource, output_location, z=0, x=0, y=0, tile_format='png'):

    # we should create a render_tile function in mapshader core which has this logic...
    if not source.is_loaded:
        print(f'Dynamically Loading Data {source.name}', file=sys.stdout)
        source.load()

    img = render_map(source, x=int(x), y=int(y), z=int(z), height=256, width=256)
    
    if np.isnan(img.data).all():
        return False
    
    try:
        import boto3
    except ImportError:
        raise ImportError('conda install boto3 to enable rendering to S3')

    try:
        from urlparse import urlparse
    except ImportError:
        from urllib.parse import urlparse
        
    # I don't think we need this but not sure...
    img = fromarray(np.flip(img.data, 0), 'RGBA') 

    s3_info = urlparse(output_location)
    bucket = s3_info.netloc
    s3_client = boto3.client('s3')

    tile_file_name = '{}.{}'.format(y, tile_format.lower())
    key = os.path.join(s3_info.path, str(z), str(x), tile_file_name).lstrip('/')
    output_buf = BytesIO()
    img.save(output_buf, tile_format)
    output_buf.seek(0)
    s3_client.put_object(Body=output_buf, Bucket=bucket, Key=key, ACL='public-read')
    return 'https://{}.s3.amazonaws.com/{}'.format(bucket, key)


to_s3_tile(cities_source, output_location='s3://mapshader-tiling-test-999/', x=1, y=1, z=1)

### Create a Pandas DataFrame of Tile to Process based on Map Source feature extents

In [ ]:
from mapshader.tile_utils import get_tiles_by_extent
import dask.dataframe as dd
import pandas as pd

# TODO: Daskify this with delayed objects...if we have billions of tiles, workers should be able to "generate" tiles as they go...


min_zoom = 0
max_zoom = 18

all_tiles = []
for i, row in cities_source.data.iterrows():
    for z in range(min_zoom, max_zoom+1):
        tiles = get_tiles_by_extent(xmin=row['buffer_0_4326_xmin'],
                                    ymin=row['buffer_0_4326_ymin'],
                                    xmax=row['buffer_0_4326_xmax'],
                                    ymax=row['buffer_0_4326_ymax'],
                                    level=z)
        for x, y, z, q in tiles:
            tile = dict(x=x, y=y, z=z, q=q)
            all_tiles.append(tile)
            
tiles_df = pd.DataFrame(all_tiles)
tiles_df.drop_duplicates().sort_values(by=['z', 'x', 'y'])

### Create Dask DataFrame and persist across cluster

In [ ]:
tiles_ddf = dd.from_pandas(tiles_df, npartitions=200)
tiles_ddf.persist()

### Map `tile to S3` helper function across tile partitions

In [ ]:
def tile_partition(df, output_location, source=None):
    
    def tile_row(row):
        _ = to_s3_tile(source,
                       output_location,
                       x=row['x'],
                       y=row['y'],
                       z=row['z'])
        return True
    
    
    return df.apply(tile_row, axis=1)

tiles_ddf.map_partitions(tile_partition,
                         source=cities_source,
                         output_location='s3://mapshader-tiling-test-999/').compute()

### View tiles on OSM basemap

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles

tiles_url = 'https://mapshader-tiling-test-999.s3.amazonaws.com/{z}/{x}/{y}.png'
tile_layer=TileLayer(url=tiles_url)

from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    zoom=4,
    scroll_wheel_zoom=True)

m.add_layer(tile_layer)
m
display(m)